# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 8 2023 10:34AM,258938,10,LUPLUP0020,"Lupin Research, Inc.",Released
1,Mar 8 2023 10:24AM,258937,10,DNMD-36021,Emerginnova,Released
2,Mar 8 2023 10:24AM,258937,10,Enova-11482,Emerginnova,Released
3,Mar 8 2023 10:22AM,258936,10,SNAP35289165,"Snap Medical Industries, LLC",Released
4,Mar 8 2023 10:22AM,258936,10,SNAP37231,"Snap Medical Industries, LLC",Released
5,Mar 8 2023 10:15AM,258897,15,PNC432566A,"Person & Covey, Inc.",Released
6,Mar 8 2023 10:15AM,258897,15,PNC432567A,"Person & Covey, Inc.",Released
7,Mar 8 2023 10:15AM,258897,15,PNC432573A,"Person & Covey, Inc.",Released
8,Mar 8 2023 10:15AM,258897,15,PNC432563A,"Person & Covey, Inc.",Released
9,Mar 8 2023 10:15AM,258896,15,30014603,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,258934,Released,4
27,258935,Released,1
28,258936,Released,2
29,258937,Released,2
30,258938,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258934,NaN,4.0
258935,NaN,1.0
258936,NaN,2.0
258937,NaN,2.0
258938,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258712,5.0,32.0
258759,16.0,15.0
258780,1.0,15.0
258828,0.0,1.0
258862,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258712,5,32
258759,16,15
258780,1,15
258828,0,1
258862,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258712,5,32
1,258759,16,15
2,258780,1,15
3,258828,0,1
4,258862,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258712,5,32
1,258759,16,15
2,258780,1,15
3,258828,,1
4,258862,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc."
1,Mar 8 2023 10:24AM,258937,10,Emerginnova
3,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC"
5,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc."
9,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc."
14,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC"
15,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.
19,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc."
35,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc."
66,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc.",,1
1,Mar 8 2023 10:24AM,258937,10,Emerginnova,,2
2,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",,2
3,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",,4
4,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",,5
5,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",,1
6,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,,4
7,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc.",1,15
8,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",16,15
9,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",5,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc.",1,
1,Mar 8 2023 10:24AM,258937,10,Emerginnova,2,
2,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2,
3,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4,
4,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5,
5,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",1,
6,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,4,
7,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc.",15,1
8,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",15,16
9,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",32,5


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc.",1,
1,Mar 8 2023 10:24AM,258937,10,Emerginnova,2,
2,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2,
3,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4,
4,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc.",1.0,NaN
1,Mar 8 2023 10:24AM,258937,10,Emerginnova,2.0,NaN
2,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2.0,NaN
3,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4.0,NaN
4,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 8 2023 10:34AM,258938,10,"Lupin Research, Inc.",1.0,0.0
1,Mar 8 2023 10:24AM,258937,10,Emerginnova,2.0,0.0
2,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2.0,0.0
3,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4.0,0.0
4,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3107102,22.0,6.0
12,1553450,5.0,1.0
15,1294044,71.0,22.0
19,517739,3.0,0.0
21,517809,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3107102,22.0,6.0
1,12,1553450,5.0,1.0
2,15,1294044,71.0,22.0
3,19,517739,3.0,0.0
4,21,517809,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,22.0,6.0
1,12,5.0,1.0
2,15,71.0,22.0
3,19,3.0,0.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,12,Released,5.0
2,15,Released,71.0
3,19,Released,3.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Executing,6.0,1.0,22.0,0.0,0.0
Released,22.0,5.0,71.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Executing,6.0,1.0,22.0,0.0,0.0
1,Released,22.0,5.0,71.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Executing,6.0,1.0,22.0,0.0,0.0
1,Released,22.0,5.0,71.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()